In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
    'rev' : ['I Loved Food in Restraunt' , 'Food was Good in Restraunt' , 'Bad Food in Restraunt'],
    'sent' : [1,1,0]
})

In [3]:
words = {
    'Loved' : 0 ,
    'Food' : 1 ,
    'Restraunt' : 2,
    'Good' : 3 ,
    'Bad' : 4
}

In [4]:
x_train = []
for i in data['rev']:
    comp = []
    for  j in i.split() :
        if words.get(j) is not None :
            arr = [0 for i in range(5)]
            arr[words.get(j)] = 1 
            comp.append(arr)
    x_train.append(comp)            

In [5]:
x_train

[[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0]],
 [[0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0]],
 [[0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0]]]

In [6]:
y_train = data['sent'].values

In [10]:
class RNN:
    def __init__(self , input_shape , nodes):
        self.nodes = nodes
        self.input_shape = input_shape
        self.weights_0 = np.random.randn(input_shape[-1] , nodes)
        self.bais_0 = np.random.randint(low = 0 , high = 1 , size = (1,nodes))
        self.recurent_weights = np.random.randn(nodes , nodes)
        self.weights_1 = np.random.randn(nodes , 1)
        self.bias_1 = np.random.randint(low = 0 , high = 1 , size = (1,1))
        self.output = None
        self.learning_rate = 0.01
        self.hidden_output_matrix = None 
        print(self.recurent_weights)
        
    def forward_propogation(self , x_train , y_train): 
        output_arr = []
        count = 0
        for i in x_train :
            hidden_output_matrix = []
            self.o = np.array([0 for i in range(self.nodes)]).reshape((1,self.nodes))
            hidden_output_matrix.append(self.o)
            for j in i :
                j = np.array(j).reshape(1,self.input_shape[1])
                self.o = 1/(1+np.exp(-(j @ self.weights_0 + self.o @ self.recurent_weights + self.bais_0)))
                hidden_output_matrix.append(self.o) 
            # Hidden Output Matrix 
            self.hidden_output_matrix = np.array(hidden_output_matrix)
            output = 1/(1+np.exp(-(self.o @ self.weights_1 + self.bias_1)))
            #  Update the Output Weights 
            self.weights_1 -= self.learning_rate * (output[0][0] - y_train[count])*(self.o.T)
            # Update the Recurent Weights
            rsum = 0
            n = self.input_shape[0]
            for i in range(n , 1 , -1):
                if i == n :
                    rsum += self.hidden_output_matrix[i]
                    prev = np.array([[1]])
                else : 
                    curr = self.hidden_output_matrix[i]@((1-self.hidden_output_matrix[i])@self.recurent_weights).T
                    rsum += prev@(curr@self.hidden_output_matrix[i-1])
                    prev = curr      
            self.recurent_weights -= self.learning_rate *(output[0][0]-y_train[count])*((1-self.o)@self.weights_1)[0][0]*(self.o).T@rsum
            output_arr.append(output[0][0])
            count += 1 
        self.output = output_arr 
        self.hidden_output_matrix = np.array(hidden_output_matrix)
        return output_arr

In [15]:
model = RNN((3,5) , 3)

[[-0.28962231  0.55502831  0.26724136]
 [-0.26202198  0.150994   -1.3362243 ]
 [-1.38790359 -0.51634478 -0.52872415]]


In [16]:
model.forward_propogation(x_train , y_train =y_train)

[0.3982589049963428, 0.40998121844232105, 0.40597958329754325]

In [17]:
model.recurent_weights

array([[-0.2926718 ,  0.55368811,  0.26913245],
       [-0.26458083,  0.14986784, -1.33458662],
       [-1.38856256, -0.51663925, -0.52833789]])

In [77]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

In [78]:
count_vectorizer = CountVectorizer(ngram_range=(1,2))

In [79]:
tf_idf_vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [82]:
x_train_arr = count_vectorizer.fit_transform(data['rev']).toarray()

In [83]:
x_train_arr_tf = tf_idf_vectorizer.fit_transform(data['rev']).toarray()

In [92]:
model2 = RNN((len(x_train_arr[0]),1) , 30)

In [94]:
model2.forward_propogation(x_train_arr)

[0.9997972345075682, 0.645344939329813, 0.001837730702261347]

In [95]:
model2.forward_propogation(x_train_arr_tf)

[0.08542275051069173, 0.5714911985863415, 0.00010105932571603357]

In [97]:
from keras.models import Sequential

In [98]:
from keras.layers import SimpleRNN , Dense

In [108]:
model3 = Sequential()
model3.add(SimpleRNN(30 , input_shape = (len(x_train_arr[0]),1) , return_sequences=True))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [115]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

call_back_history = model3.fit(x_train_arr, data['sent'], epochs=10)


Epoch 1/10
1/1 [==============================] - 0s 97ms/step - loss: 0.1914 - accuracy: 0.9762
Epoch 2/10
1/1 [==============================] - 0s 65ms/step - loss: 0.1859 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 88ms/step - loss: 0.1808 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 80ms/step - loss: 0.1759 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 64ms/step - loss: 0.1714 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 81ms/step - loss: 0.1670 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 73ms/step - loss: 0.1629 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 81ms/step - loss: 0.1590 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 100ms/step - loss: 0.1552 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 80ms/step - loss: 0.1517 - accuracy: 1.0000


In [ ]:
import matplotlib.pyplot asplt

In [119]:
plt.plot(call_back_history.history['accuracy']

[0.9761905074119568, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]